# Species Distribution Model (SDM)

## Import Libraries

In [9]:
#Libraries
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

#Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Modeling & Metrics
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Load the Dataset

In [10]:
# Species data
df = pd.read_csv('../all_species.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949 entries, 0 to 1948
Columns: 160 entries, Datetime to Abundance (ind/m2)
dtypes: float64(153), int64(2), object(5)
memory usage: 2.4+ MB


In [11]:
df.sort_values(by='Year', inplace=True) 
df.reset_index(inplace=True) # reset index for proper min/max time
df = df[df['Zone'] != 'F'] # retroactively remove Zone F which is absent in training set
df

,index,Datetime,Year,Month,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,...,Callionymus lira (peixe-pau lira),Oncidiella celtica,Doriopsilla areolata (nudibrânquio),Scorpaena sp. (Rascasso),Lipophrys pholis (ad.),Diplodus cervinus,Gobiusculus flavescens,Sessile Coverage,Total Mobile Species,Abundance (ind/m2)
0,1948,11/28/2011,2011,11,0.6,Clear sky,16.0,D,Medium,Puddle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.45,4.0,0.05
1,1923,12/12/2011,2011,12,0.9,Clear sky,16.0,E,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.00,8.0,0.00
2,1922,12/12/2011,2011,12,0.9,Clear sky,16.0,E,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.25,0.0,0.00
3,1921,12/12/2011,2011,12,0.9,Clear sky,16.0,E,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4.0,0.00
4,1920,12/12/2011,2011,12,0.9,Clear sky,16.0,E,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,41,6/5/2020,2020,6,0.6,Sunny,19.0,E,Medium,Puddle/Rock/Sand,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.05,2.0,0.10
1944,47,5/8/2020,2020,5,0.4,Sunny,17.0,D,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.65,2.0,0.15
1945,48,5/8/2020,2020,5,0.4,Sunny,17.0,D,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.00,2.0,0.15
1946,49,5/8/2020,2020,5,0.4,Sunny,17.0,D,Medium,Rock,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.50,2.0,0.15


In [12]:
df[['Tide', 'Water temperature (ºC)', 'Sessile Coverage', 'Total Mobile Species','Abundance (ind/m2)']].describe()

,Tide,Water temperature (ºC),Sessile Coverage,Total Mobile Species,Abundance (ind/m2)
count,1879.000000,1879.000000,1879.000000,1875.000000,1879.000000
mean,0.729159,16.821767,52.711895,3.851733,0.215540
std,0.178576,2.266078,34.378994,12.931326,0.657095
min,0.300000,11.000000,0.000000,0.000000,0.000000
25%,0.600000,15.000000,21.025000,0.000000,0.000000
50%,0.700000,17.000000,56.500000,0.000000,0.000000
75%,0.900000,19.000000,84.900000,3.000000,0.150000
max,1.400000,22.000000,123.500000,254.000000,12.700000


**Initial Observations**
- `Tide` has a nearly equal mean and median with a majority of values spread within 2 (TODO: How was tide measured?), indicating a possible normal distribution.
- `Water temperature (ºC)` may have a similar distribution to `Tide`. Are observed min and max values related for these features due to an event?
- `Sessile Coverage` may need to be plotted to confirm if the distribution is normal. Is there a time factor, like seasonality?
- `Total Mobile Species` and related field `Abundance(ind/m2)` has a relatively large range of sample values. Double check that thes values appear to be correlated.

**Note**: This evaluation is not only to determine the shape of the distribution, as all numeric columns are transformed by removing the mean value of each feature, then scaling it using SciKit Learn's Preprocessing library.


## Data Preprocessing

#### Categorical Features

In [13]:
df["Weather Condition"].value_counts()

Clear sky          1380
Cloudy              320
Rain                 83
Sunny                59
Fairly Cloudy        33
Sunny and Windy       4
Name: Weather Condition, dtype: int64

In [14]:
df["Weather Condition"].replace(to_replace="Sunny and Windy", value="Sunny", inplace=True)

In [15]:
df["Weather Condition"].value_counts()

Clear sky        1380
Cloudy            320
Rain               83
Sunny              63
Fairly Cloudy      33
Name: Weather Condition, dtype: int64

#### Datetime

In [16]:
df['Datetime'] = df.loc[:, 'Datetime'].astype('datetime64[ns]')

In [17]:
min_date = df.Datetime.iloc[0]
max_date = df.Datetime.iloc[-1]
print("Min:", min_date, "Max:", max_date)

Min: 2011-11-28 00:00:00 Max: 2020-11-16 00:00:00


### Building a Pipeline
Note: Numerical features are considered here. 

In [18]:
numeric_features = ['Tide', 'Water temperature (ºC)', 'Sessile Coverage']
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), 
           ("scaler", StandardScaler())]
)

datetime_features = ['Month', 'Year']
categorical_features = ['Weather Condition', 'Zone']
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features + datetime_features),
    ]
)

### Train/Test split

Train percent determined based on a Discord discussion regarding the "Abundance measure inconsistency around Septemper 2015":

"From the information of that year’s report, there was a damage to the pier holding the sand at the protected area in the storm of 2014 and continued in 2015, this caused the increase in sand in the rocky shore and therefore the decrease in abundance. The pier was re-established in the Summer of 2016."

In [19]:
# Adapted from https://www.rasgoml.com/feature-engineering-tutorials/scikit-learn-time-series-split

# Determine temporal split
train_percent = .5
time_between = max_date - min_date
train_cutoff = min_date + train_percent*time_between
print("Train Cutoff Date:", train_cutoff)

Train Cutoff Date: 2016-05-23 00:00:00


In [20]:
# Define X and y
train_df = df.loc[df['Datetime'] <= train_cutoff]
test_df = df.loc[df['Datetime'] > train_cutoff]

In [21]:
print("Train:", train_df.Zone.unique())
print("Test:", test_df.Zone.unique())
print("Train:", train_df['Datetime'].min(), train_df['Datetime'].max())
print("Test:", test_df['Datetime'].min(), test_df['Datetime'].max())


Train: ['D' 'E' 'B' 'A']
Test: ['B' 'A' 'D' 'E']
Train: 2011-11-28 00:00:00 2016-05-10 00:00:00
Test: 2016-06-07 00:00:00 2020-11-16 00:00:00


In [22]:
train_df.set_index('Datetime', inplace=True)
test_df.set_index('Datetime', inplace=True)

In [23]:
X_train = train_df[numeric_features + categorical_features + datetime_features]
y_train = train_df['Abundance (ind/m2)']
X_test = test_df[numeric_features + categorical_features + datetime_features]
y_test = test_df['Abundance (ind/m2)']
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (1318, 7)
y_train: (1318,)
X_test: (561, 7)
y_test: (561,)


## Model Selection

#### Linear Regressor

In [24]:
# Create the pipeline with preprocessor and linear regressor
linear_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear', LinearRegression())
])

# Fit the pipeline on the data
linear_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Tide',
                                                   'Water temperature (ºC)',
                                                   'Sessile Coverage']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Weather Condition', 'Zone',
                                                   'Month', 'Year'])])),
                ('linear', LinearRegression())])

#### Random Forest Regressor

In [25]:
# Create the pipeline with preprocessor and random forest regressor
forest_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('forest', RandomForestRegressor(random_state = 0))
])

# Fit the pipeline on the data
forest_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Tide',
                                                   'Water temperature (ºC)',
                                                   'Sessile Coverage']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Weather Condition', 'Zone',
                                                   'Month', 'Year'])])),
                ('forest', RandomForestRegressor(random_state=0))])

### Model Evaluation
Evaluate the trained models using appropriate metrics such as mean squared error (MSE) and mean absolute error (MAE).  Compare the performance of different models.

In [29]:
linear_y_preds = linear_pipeline.predict(X_test)
linear_mse = mean_squared_error(y_test, linear_y_preds)
linear_rmse = linear_mse**0.5
print("MSE: ", linear_mse)
print("RMSE: ", linear_rmse)

MSE:  0.0988731550919198
RMSE:  0.31444102005291835


In [32]:
# Predict and score
forest_y_preds = forest_pipeline.predict(X_test)
forest_mse = mean_squared_error(y_test, forest_y_preds)
forest_rmse = forest_mse**0.5
print("MSE: ", forest_mse)
print("RMSE: ", forest_rmse)

MSE:  0.13934587826074674
RMSE:  0.37329060832111316


### Feature Importances
Determine features that influence the abundance of mobile species.

In [33]:
feature_names = forest_pipeline.named_steps['preprocessor'].get_feature_names_out()
importances = forest_pipeline.named_steps['forest'].feature_importances_
std = np.std([tree.feature_importances_ for tree in forest_pipeline.named_steps['forest'].estimators_], axis=0)

In [34]:
forest_importances = pd.Series(importances, index=feature_names)
forest_importances = forest_importances.sort_values(ascending=False)

In [35]:
# plot the impurity-based importance

fig = px.bar(forest_importances[:5], title="Feature importances using MDI")
fig.show()

[Source](https://towardsdatascience.com/extracting-feature-importances-from-scikit-learn-pipelines-18c79b4ae09a)

### Test Model

In [36]:
# Create dataframe for plotting results

fitted_values = pd.DataFrame(forest_y_preds, columns=['Abundance (ind/m2)'], index=y_test.index).reset_index()
fitted_values['Datetime'] = fitted_values['Datetime'].astype('datetime64[ns]')
fitted_values = fitted_values.groupby(fitted_values['Datetime'].dt.year).mean()

actual_values = pd.DataFrame(y_test).reset_index()
actual_values = actual_values.groupby(actual_values['Datetime'].dt.year).mean()

# Create identifiers
fitted_values['Type'] = "Fitted Value"
actual_values['Type'] = "Actual Value"

# merge dataframes
results = pd.concat([fitted_values, actual_values])
results.reset_index(inplace=True)

/var/folders/7l/441dnk_52sgfldpwf_5_7dmr0000gn/T/ipykernel_15015/96204431.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/7l/441dnk_52sgfldpwf_5_7dmr0000gn/T/ipykernel_15015/96204431.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [40]:
px.line(results, x='Datetime', y='Abundance (ind/m2)', color='Type')

### Model Prediction

In [43]:
X_train['Zone'].value_counts()

A    416
E    362
D    300
B    240
Name: Zone, dtype: int64

In [92]:
df.loc[df['Month'] == 12, ('Water temperature (ºC)', 'Sessile Coverage', 'Tide')].describe()

,Water temperature (ºC),Sessile Coverage,Tide
count,153.000000,153.000000,153.000000
mean,15.266667,52.270915,0.748235
std,0.720837,32.679647,0.192252
min,13.300000,0.000000,0.400000
25%,15.000000,24.000000,0.520000
50%,15.000000,54.600000,0.900000
75%,16.000000,81.000000,0.900000
max,16.000000,99.450000,1.000000


In [89]:
toy_data = {'Datetime':'2020-12-18', 'Tide':0.8, 'Water temperature (ºC)': 16, 'Sessile Coverage': 60.0, 'Weather Condition':'Clear sky', 'Zone':'A', 'Month':12, 'Year':2020}
toy_df = pd.Series(toy_data)
toy_df

Datetime                  2020-12-18
Tide                             0.8
Water temperature (ºC)            16
Sessile Coverage                60.0
Weather Condition          Clear sky
Zone                               A
Month                             12
Year                            2020
dtype: object

In [93]:
# Predicting abundance for rising tides and temperatures
# Sessile coverage 8% above average
predicted_abundance = forest_pipeline.predict(pd.DataFrame(toy_df).T)
print("Predicted Abundace(ind/m2) for 2020-12-18: ", predicted_abundance)
print("Distance from last actual observation value for 2020-11-16: ", predicted_abundance - 0.0736)
print("Distance from last fitted value for 2020-11-16: ", predicted_abundance - 0.3269)

array([0.0854])

The model had predicted that mobile species would have an abundance value of 0.33 for one of the final dates in November 2020. To test the affect of temperatures and tides on species occurences, data was fabricated to model conditions in December 2020 if they were above average values for the tide and water temperature. Additionally, the test data models a higher percentage of sessile coverage in Zone. 

The result of this test was a predicted value that fell below the predicted trend line by 0.17 units and above the actual observed value by 0.08 units (as seen above). By adjusting the conditions to reflect above above average measurements of ecological features, the model was able to predict more accurate results. This is in spite of a 0.25 unit difference between the actual value and fitted value the predicted with using the real test data. 

This all suggests that the unexpected recovery trends may be a result of rising tides, water temperatures, and sessile coverage.